In [1]:
import pandas as pd
import numpy as np

import os, glob
import re

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/4222_tesla/scrape')

Mounted at /content/drive


In [2]:
keyword_dict = {'tm3':['#model3', 'model 3', 'tesla model 3'], 
                'tmx':['#modelx', 'model x', 'tesla model x'],
                'tmy':['#modely', 'model y', 'tesla model y'],
                'tms':['#models', 'model s', 'tesla model s']}

all_files = glob.glob(os.path.join(os.getcwd(), 't*.csv'))
all_files = [filename.split('/')[-1] for filename in all_files]
all_files

['tm3_jan_apr.csv', 'tmx_jan_apr.csv', 'tmy_jan_apr.csv', 'tms_jan_apr.csv']

In [3]:
# 1. Drop unnecessary columns
def get_req_col(df_in):
    return df_in.iloc[:,[3,4,7,10,15,16,17]]
#df2 = df2.to_csv(f"dropped_Tesla_contents.csv", index=False) 

In [4]:
# 2. Clean tweets and drop duplicates (added replace '  ' with ' ')
def clean_df(df_in):
    df_out = df_in.copy()
    df_out['tweet'] = df_out['tweet'].str.replace("\$[^\s]+", '').str.replace("'", '').str.replace("@[^\s]+", '').str.replace("#[^\s]+", '').str.replace("https[^\s]+", '').str.replace("[^\x00-\x7F]+", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()
    df_out.drop_duplicates(subset = ['tweet'])
    return df_out

In [7]:
# 3. Drop rows where tweet does not contain keywords (works better after fixing double spacing)
#    df_in = pd.read_csv(df_path)
def drop_rows(df_in, keywords):
    to_drop = []
    for row_index, row in df_in.iterrows():
        if keywords[0] not in row['tweet'].lower() and keywords[1] not in row['tweet'].lower() and keywords[2] not in row['tweet'].lower():
            to_drop.append(row_index)
    df_out = df_in.drop(to_drop)
    return df_out

In [11]:
for filename in all_files:
    product = filename.split('_')[0]
    keywords = keyword_dict[product]
    df = pd.read_csv(filename)
    print('Now processing:', product)
    print('Keywords:', keywords)
    print('Start len:', len(df))
    # Keep required columns
    df_req_col = get_req_col(df)
    # Clean text in tweets
    df_req_col_clean = clean_df(df_req_col)
    # Drop tweets which do not contain keywords (Scraped cos keywords found in username etc.)
    df_req_col_clean_dropped = drop_rows(df_req_col_clean, keywords)
    print('End len:', len(df_req_col_clean_dropped))
    # Save as tm3.csv, etc.
    save_path = f'/drive/My Drive/folder_name/processed_{product}.csv'
    df_req_col_clean_dropped.to_csv(f'processed_{product}.csv', index=False) 


Now processing: tm3
Keywords: ['#model3', 'model 3', 'tesla model 3']
Start len: 5721


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


End len: 5056
Now processing: tmx
Keywords: ['#modelx', 'model x', 'tesla model x']
Start len: 2778
End len: 2478
Now processing: tmy
Keywords: ['#modely', 'model y', 'tesla model y']
Start len: 3527
End len: 2888
Now processing: tms
Keywords: ['#models', 'model s', 'tesla model s']
Start len: 910
End len: 512
